In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
from matplotlib import cm

def find_nearest(array, value):
    idx = (np.abs(array - value)).argmin()
    return idx

In [6]:
def Reflectivity_S(theta_i,n_i,n_t):
	if n_t**2-(n_i*np.sin(theta_i))**2 >= 0:
		cos_theta_t = 1/n_t*np.sqrt(n_t**2-(n_i*np.sin(theta_i))**2)
	else:
		cos_theta_t = -1j*1/n_t*np.sqrt((n_i*np.sin(theta_i))**2-n_t**2)

	return (n_i*np.cos(theta_i)-n_t*cos_theta_t)/(n_i*np.cos(theta_i)+n_t*cos_theta_t)

def Reflectivity_P(theta_i,n_i,n_t):
	if n_t**2-(n_i*np.sin(theta_i))**2 >= 0:
		cos_theta_t = 1/n_t*np.sqrt(n_t**2-(n_i*np.sin(theta_i))**2)
	else:
		cos_theta_t = -1j*1/n_t*np.sqrt((n_i*np.sin(theta_i))**2-n_t**2)
	return (n_i*cos_theta_t-n_t*np.cos(theta_i))/(n_i*cos_theta_t+n_t*np.cos(theta_i))

def Transmission_S(theta_i,n_i,n_t):
	if n_t**2-(n_i*np.sin(theta_i))**2 >= 0:
		cos_theta_t = 1/n_t*np.sqrt(n_t**2-(n_i*np.sin(theta_i))**2)
	else:
		cos_theta_t = -1j*1/n_t*np.sqrt((n_i*np.sin(theta_i))**2-n_t**2)
	return 2*n_i*np.cos(theta_i)/(n_i*np.cos(theta_i)+n_t*cos_theta_t)

def Transmission_P(theta_i,n_i,n_t):
	if n_t**2-(n_i*np.sin(theta_i))**2 >= 0:
		cos_theta_t = 1/n_t*np.sqrt(n_t**2-(n_i*np.sin(theta_i))**2)
	else:
		cos_theta_t = -1j*1/n_t*np.sqrt((n_i*np.sin(theta_i))**2-n_t**2)
	return 2*n_i*np.cos(theta_i)/(n_i*cos_theta_t+n_t*np.cos(theta_i))

def Interface(theta_i,n_i,n_t,pol):
	if pol == 's':		
		reflection = [(1,Reflectivity_S(theta_i,n_i,n_t)),(Reflectivity_S(theta_i,n_i,n_t),1)]
		return reflection/Transmission_S(theta_i,n_i,n_t)
	else:
		reflection = np.array([(1,Reflectivity_P(theta_i,n_i,n_t)),(Reflectivity_P(theta_i,n_i,n_t),1)])
		return 1.0/Transmission_P(theta_i,n_i,n_t)*reflection

def Phasor(theta,lbd,L,n):
	delta = (2*np.pi/lbd)*n*L*np.cos(theta)
	return np.array([(np.exp(-1j*delta),0),(0,np.exp(1j*delta))])

In [ ]:
def get_Quality(lbd,R):
	Rmax = max(R)
	Rmin = min(R)

	max_ind = find_nearest(R,Rmax)
	min_ind = find_nearest(R,Rmin)

	left_half = find_nearest(R[:int(np.size(R)/2)],(Rmax-Rmin)/2+Rmin)
	right_half = int(np.size(R)/2)+find_nearest(R[int(np.size(R)/2):-1],(Rmax-Rmin)/2+Rmin)

	FWHM = np.abs(lbd[left_half]-lbd[right_half])
	return lbd[min_ind]/FWHM

def get_Resonance(lbd, R):
    pass

In [ ]:
class Layer():
	def __init__(self, n, h, num):
		self.n = n
		self.h = h
		self.num = num

class Stack():
	def __init__(self, n, layer1, layer2):
		self.layers = []
		for i in range(2*n):
			self.layers.append(layer1)
			self.layers.append(layer2)


def TMM(lbd=None, lbd_res=None, theta_init=None, N_layers=None, n_origin=None, n_stack=None, n_Substrate=None, polarization=None, cav=False, cav_pos=0):
	M = np.array([(1+0j,0),(0,1+0j)])
		
	M = np.dot(Interface(theta_init,n_origin,n_stack[0],polarization),M)	
	theta_init = np.arcsin(n_origin/n_stack[1]*np.sin(theta_init)) if abs(n_origin/n_stack[0]*np.sin(theta_init))<=1 else 0	
	
	if cav:
		num_stacks = 2*N_layers+1
	else:
		num_stacks = 2*N_layers
	
	for i in range(0,num_stacks):

		if cav and i==2*cav_pos:
			hC = lbd_res/3.56
			M = np.dot(Phasor(theta_init,lbd,hC,3.56),M)				
			M = np.dot(Interface(theta_init,3.56,3.05,polarization),M)		
			theta_init = np.arcsin(3.05/3.56*np.sin(theta_init)) if abs(3.05/3.56*np.sin(theta_init))<=1 else 0
			continue
		
		M = np.dot(Phasor(theta_init,lbd,lbd_res/(4*n_stack[i%2]),n_stack[i%2]),M)				
		M = np.dot(Interface(theta_init,n_stack[(i)%2],n_stack[(i+1)%2],polarization),M)	
		theta_init = np.arcsin(n_stack[i%2]/n_stack[(i+1)%2]*np.sin(theta_init)) if abs(n_stack[i%2]/n_stack[(i+1)%2]*np.sin(theta_init))<=1 else 0


	R = np.abs(M[1,0]/M[0,0])**2
	return R

thetas = np.linspace(0,30*np.pi/180,100)
thetas = [0]
# plt.plot(180/np.pi*thetas, [Reflectivity_S(theta,3.05,3.56)**2 for theta in thetas],'-r',180/np.pi*thetas, [Reflectivity_P(theta,3.05,3.56)**2 for theta in thetas],'-b')
# plt.plot(180/np.pi*thetas, [Reflectivity_S(theta,3.56,3.05)**2 for theta in thetas],'+r',180/np.pi*thetas, [Reflectivity_P(theta,3.56,3.05)**2 for theta in thetas],'+b')
# plt.show()
#thetas = [np.arctan(2.4048/(0.2e-6*3.56)*930e-9/(2*np.pi)),np.arctan(2.4048/(0.6e-6*3.56)*930e-9/(2*np.pi))]
lbd = np.linspace(929.95,930.05,10000)

NT = np.linspace(9,9,1, dtype=np.int)
Q = []

R_S = np.zeros((np.size(thetas),np.size(lbd)))
R_P = np.zeros((np.size(thetas),np.size(lbd)))

for nt in NT:
	R_S[0,:] = TMM(lbd=lbd, lbd_res=930, theta_init=0, 
		N_layers=52, n_origin=1, n_stack=[3.45,2.73], n_Substrate=2.96, polarization='s', cav=True, cav_pos=16)
	Q.append(findQ(lbd,R_S[0]))	
	plt.plot(lbd,R_S[0])
	plt.show()
	print(Q[-1])
	# R_P[i,:] = TMM(lbd=lbd, lbd_res=930, theta_init=theta, 
	# 	N_layers=[22,24], n_origin=1, n_stack=[3.56,3.05], n_Substrate=3.56, polarization='p')

In [13]:
import scipy.special as sp

nu = 50e9
root = 2.4048
c_l = 18300
c_t = 11500
v_eff = (1/3*1/c_l**3 + 2/3*1/c_t**3)**(-1/3)
lbd = v_eff/nu
R = root*lbd/(2*np.pi)

2.532166178568285e-07